# 1、LSTM+CNN+RNN调参

## （1）准备数据

In [2]:
!wget https://github.com/kumarvinit15/Data-Science-Capstone/archive/master.zip

--2020-08-14 09:17:38--  https://github.com/kumarvinit15/Data-Science-Capstone/archive/master.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/kumarvinit15/Data-Science-Capstone/zip/master [following]
--2020-08-14 09:17:38--  https://codeload.github.com/kumarvinit15/Data-Science-Capstone/zip/master
Resolving codeload.github.com (codeload.github.com)... 140.82.112.10
Connecting to codeload.github.com (codeload.github.com)|140.82.112.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [ <=>                ]   1.83M  --.-KB/s    in 0.1s    

2020-08-14 09:17:38 (17.1 MB/s) - ‘master.zip’ saved [1921556]



In [3]:
!unzip master.zip

Archive:  master.zip
0c007d4e0d08cd4c6e6e92aca2dc4d2ba20877d8
   creating: Data-Science-Capstone-master/
  inflating: Data-Science-Capstone-master/DS Capstone.ipynb  
  inflating: Data-Science-Capstone-master/README.md  
  inflating: Data-Science-Capstone-master/sample_submission.csv  
  inflating: Data-Science-Capstone-master/test.csv  
  inflating: Data-Science-Capstone-master/train.csv  


In [ ]:
!pip install emoji
!pip install fasttext

In [1]:
from google.colab import drive  
drive.mount('/content/gdrive') 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [1]:
import numpy as np 
import pandas as pd 
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub
import emoji
import os
import time
import sys
import fasttext
import re
import nltk 
nltk.download('punkt')
from nltk.corpus import stopwords
import tensorflow as tf
tf.keras.backend.clear_session()

import itertools
import collections
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
#load data 
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

ModuleNotFoundError: No module named 'tensorflow_hub'

In [5]:
import numpy as np
import pandas as pd 
train = pd.read_csv("Data-Science-Capstone-master/train.csv")
train.head()
print(train['keyword'].isna())

0       True
1       True
2       True
3       True
4       True
        ... 
7608    True
7609    True
7610    True
7611    True
7612    True
Name: keyword, Length: 7613, dtype: bool


In [ ]:
test = pd.read_csv("Data-Science-Capstone-master/test.csv")
test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [ ]:
train=train[['text','target']]
test=test[['id','text']]

In [ ]:
# check class distribution in train dataset
from scipy import stats
train.groupby(['target']).size()

target
0    4342
1    3271
dtype: int64

In [ ]:
x_train = list(train['text'])  # 转化为列表的形式

## （2）分词

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)
word_index = tokenizer.word_index
data = pad_sequences(sequences, maxlen=100)

labels = np.asarray(train['target'])  # 预处理

In [ ]:
training_samples = 5200

x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data[training_samples: ]
y_val = labels[training_samples: ]

## （3）下载Glove词嵌入

In [ ]:
!wget http://nlp.stanford.edu/data/wordvecs/glove.6B.zip

--2020-08-14 03:08:42--  http://nlp.stanford.edu/data/wordvecs/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/wordvecs/glove.6B.zip [following]
--2020-08-14 03:08:42--  https://nlp.stanford.edu/data/wordvecs/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.6B.zip [following]
--2020-08-14 03:08:42--  http://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182753 (822M) [applic

In [ ]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       
  inflating: glove.6B.50d.txt        


In [ ]:
import numpy as np
glove_dir = ''

embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [ ]:
embedding_dim = 100  # 这里的维度必须为100

embedding_matrix = np.zeros((10000, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if i < 100:
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector  # 嵌入索引（embeddings_index） 中找不到的词，其嵌入向量全为 0

## （4）模型

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense
from keras.layers import LSTM
from keras.layers import SimpleRNN

model = Sequential()
model.add(Embedding(10000, embedding_dim, input_length=100))
model.add(LSTM(64))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(5, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 100)          1000000   
_________________________________________________________________
lstm (LSTM)                  (None, 64)                42240     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                330       
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 55        
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 6

In [ ]:
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

In [ ]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])
history = model.fit(x_train, y_train,
                    epochs=100,
                    batch_size=32,
                    validation_split=0.1)
model.save_weights('pre_trained_glove_model.h5')

Epoch 1/100
147/147 [==============================] - 1s 10ms/step - loss: 0.6366 - acc: 0.6415 - val_loss: 0.6278 - val_acc: 0.6712
Epoch 2/100
147/147 [==============================] - 1s 7ms/step - loss: 0.6017 - acc: 0.6850 - val_loss: 0.6186 - val_acc: 0.6538
Epoch 3/100
147/147 [==============================] - 1s 7ms/step - loss: 0.5866 - acc: 0.6985 - val_loss: 0.6453 - val_acc: 0.6712
Epoch 4/100
147/147 [==============================] - 1s 7ms/step - loss: 0.5753 - acc: 0.7058 - val_loss: 0.6620 - val_acc: 0.6423
Epoch 5/100
147/147 [==============================] - 1s 7ms/step - loss: 0.5660 - acc: 0.7111 - val_loss: 0.6205 - val_acc: 0.6538
Epoch 6/100
147/147 [==============================] - 1s 7ms/step - loss: 0.5602 - acc: 0.7128 - val_loss: 0.6939 - val_acc: 0.6519
Epoch 7/100
147/147 [==============================] - 1s 7ms/step - loss: 0.5549 - acc: 0.7175 - val_loss: 0.6362 - val_acc: 0.6904
Epoch 8/100
147/147 [==============================] - 1s 7ms/step -

## （5）评价

In [ ]:
test = pd.read_csv("Data-Science-Capstone-master/train.csv")
test = test[['text','target']]


In [ ]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(test['text'])
sequences = tokenizer.texts_to_sequences(test['text'])
word_index_2 = tokenizer.word_index
x_test = pad_sequences(sequences, maxlen=100)

y_test = np.asarray(test['target'])  

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print('损失'+str(test_loss))
print('测试集精度'+str(test_acc))

238/238 [==============================] - 1s 3ms/step - loss: 1.9067 - acc: 0.8365
损失1.9067246913909912
测试集精度0.8364639282226562


In [ ]:
pre = pd.read_csv("Data-Science-Capstone-master/test.csv")

pre = pre[['text']]

In [ ]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(pre['text'])
sequences = tokenizer.texts_to_sequences(pre['text'])
word_index_2 = tokenizer.word_index
x_pre = pad_sequences(sequences, maxlen=100)

# y_test = np.asarray(test['target'])  

In [ ]:

y = model.predict(x_pre)
y=np.arrray(y)
for i in range(0, len(y)):
  if y[i]>=0.5:
    y[i]=int('1')
  else:
    y[i]=int('0')
print(y)

[[0.]
 [1.]
 [0.]
 ...
 [0.]
 [0.]
 [1.]]


In [ ]:
sub=pd.read_csv("Data-Science-Capstone-master/sample_submission.csv")
sub['target']=y
#print(sub)
sub.head()
sub.to_csv("sub.csv",index=False)